<a href="https://colab.research.google.com/github/ProAek11/StrategyTradingFrendersis-AI-System/blob/main/Trade_Management_BollingerBand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import yfinance as yf

dfSPY = pd.read_csv("SPY.USUSD_Candlestick_1_D_BID_16.02.2017-21.05.2022.csv")

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY = dfSPY.reset_index()
dfSPY.head()

,index,Gmt time,Open,High,Low,Close,Volume
0,0,15.02.2017 22:00:00.000,234.580,234.948,233.85,234.726,58.1260
1,1,16.02.2017 22:00:00.000,233.945,235.060,233.93,235.045,63.6515
2,5,20.02.2017 22:00:00.000,235.527,236.679,235.51,236.406,61.4310
3,6,21.02.2017 22:00:00.000,236.006,236.530,235.83,236.277,57.9699
4,7,22.02.2017 22:00:00.000,236.879,236.890,235.56,236.438,66.0937


In [ ]:
import pandas_ta as ta
dfSPY['EMA']=ta.sma(dfSPY.Close, length=200)#sma ema
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=2)
#dfSPY.ta.indicators()
#help(ta.bbands)
my_bbands = ta.bbands(dfSPY.Close, length=20, std=2.5)
my_bbands[0:50]
dfSPY=dfSPY.join(my_bbands)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
#dfSPY.set_index('Gmt time', inplace=True)
dfSPY[420:425]

,level_0,index,Gmt time,Open,High,Low,Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5
420,619,900,04.08.2019 21:00:00.000,288.048,288.187,281.707,283.987,492.8170,278.697720,0.387511,288.239307,298.10885,307.978393,6.621436,-0.215426
421,620,901,05.08.2019 21:00:00.000,285.868,288.008,284.258,287.767,432.4100,278.734765,41.144022,286.268408,297.63480,309.001192,7.637811,0.065922
422,621,902,06.08.2019 21:00:00.000,284.378,288.808,282.048,288.008,473.0290,278.793270,44.062405,284.610883,297.10580,309.600717,8.411089,0.135940
423,622,903,07.08.2019 21:00:00.000,289.588,293.537,288.997,293.528,345.3303,278.879925,82.901298,284.243244,296.81830,309.393356,8.473235,0.369174
424,623,904,08.08.2019 21:00:00.000,292.538,293.228,289.631,291.598,353.9363,278.962675,55.806074,283.685694,296.36580,309.045906,8.557064,0.311997


In [ ]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY, 3)

In [ ]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if df.EMASignal[i]==2 and df.Close[i]<=df['BBL_20_2.5'][i]:# and df.RSI[i]<=100: #Added RSI condition to avoid direct close condition
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.EMASignal[i]==1 and df.Close[i]>=df['BBU_20_2.5'][i]:# and df.RSI[i]>=0:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal

addorderslimit(dfSPY, 0.01)

In [ ]:
dfSPY[dfSPY.ordersignal!=0]

,level_0,index,Gmt time,Open,High,Low,Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,EMASignal,ordersignal
44,243,354,04.02.2018 22:00:00.000,273.448,275.838,263.887,264.317,390.6229,253.122965,0.600541,266.639750,279.05205,291.464350,8.896047,-0.093566,2,261.67383
47,246,357,07.02.2018 22:00:00.000,267.957,268.148,257.797,257.817,280.6260,253.543565,8.604211,259.404823,277.64905,295.893277,13.141933,-0.043516,2,255.23883
76,275,399,21.03.2018 21:00:00.000,267.878,268.858,263.358,263.682,79.2601,258.155925,1.577057,263.911133,273.07390,282.236667,6.710833,-0.012503,2,261.04518
216,415,601,09.10.2018 21:00:00.000,286.818,286.837,277.868,278.088,370.4242,276.208145,0.165167,282.151322,289.84885,297.546378,5.311409,-0.263937,2,275.30712
362,561,816,12.05.2019 21:00:00.000,282.388,283.468,279.917,280.948,407.6652,277.265285,8.146933,282.754791,290.46840,298.182009,5.311152,-0.117117,2,278.13852
419,618,897,01.08.2019 21:00:00.000,293.837,294.088,290.887,292.598,416.2207,278.679670,1.834351,293.004309,298.75090,304.497491,3.847079,-0.035352,2,289.67202
420,619,900,04.08.2019 21:00:00.000,288.048,288.187,281.707,283.987,492.8170,278.697720,0.387511,288.239307,298.10885,307.978393,6.621436,-0.215426,2,281.14713
461,660,958,01.10.2019 21:00:00.000,291.497,291.497,286.627,288.058,169.1553,283.394955,5.598242,290.109403,297.90775,305.706097,5.235411,-0.131528,2,285.17742
560,759,1104,24.02.2020 22:00:00.000,323.917,324.588,311.687,312.757,272.9597,304.102270,0.750071,314.287968,330.88225,347.476532,10.030325,-0.046129,2,309.62943
732,931,1350,27.10.2020 21:00:00.000,332.098,332.838,326.118,326.537,1609.8844,312.419230,2.003795,327.200090,342.23550,357.270910,8.786587,-0.022051,2,323.27163


# Visualization

In [ ]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:1250].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.5'], line=dict(color='blue', width=1), name="BBL_20_2.5"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.5'], line=dict(color='blue', width=1), name="BBU_20_2.5")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [ ]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [ ]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.05
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()

        for j in range(0, len(self.orders)):
            if self.data.index[-1]-self.ordertime[0]>10:#days max to fulfill the order!!!
                self.orders[0].cancel()
                self.ordertime.pop(0)

        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)

            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()

        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/100, commission=.000)
stat = bt.run()
stat

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5560\2893711935.py:47: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    1125.0
Duration                               1125.0
Exposure Time [%]                    3.285968
Equity Final [$]                  20646.18338
Equity Peak [$]                   20867.95993
Return [%]                         106.461834
Buy & Hold Return [%]               47.141288
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.104927
Avg. Drawdown [%]                   -5.682418
Max. Drawdown Duration                  356.0
Avg. Drawdown Duration              62.888889
# Trades                                  9.0
Win Rate [%]                        88.888889
Best Trade [%]                       3.131086
Worst Trade [%]                     -0.213436
Avg. Trade [%]                    

In [ ]:
bt.plot(show_legend=False)

Row(id='24301', ...)

In [ ]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,381,45,50,259.77700,263.997,1607.82000,0.016245,45,50,5
1,440,77,79,261.04518,266.167,2253.60080,0.019620,77,79,2
2,498,217,221,275.30712,280.398,2535.25824,0.018492,217,221,4
3,583,373,378,278.13852,282.288,2419.14684,0.014919,373,378,5
4,646,420,424,288.04800,292.538,2900.54000,0.015588,420,424,4
5,753,462,464,285.17742,293.458,6235.27674,0.029037,462,464,2
6,906,562,565,305.42700,309.468,3661.14600,0.013231,562,565,3
7,968,734,736,323.27163,333.697,10091.75816,0.032250,734,736,2
8,1107,819,820,372.93993,384.617,12926.51649,0.031311,819,820,1
9,1259,964,966,429.30954,432.976,4616.07314,0.008540,964,966,2


In [ ]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 0.05 #1000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2.
        perc = 0.02

        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            sl1 = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]+sldiff*TPSLRatio
            tp2 = self.data.Close[-1]+sldiff
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            sl1 = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]-sldiff*TPSLRatio
            tp2 = self.data.Close[-1]-sldiff
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/100, commission=0.0000)
stat = bt.run()
stat

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5560\2361615614.py:31: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    1125.0
Duration                               1125.0
Exposure Time [%]                   10.568384
Equity Final [$]                  18123.23474
Equity Peak [$]                   25295.94574
Return [%]                          81.232347
Buy & Hold Return [%]               47.141288
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -37.044089
Avg. Drawdown [%]                  -13.760854
Max. Drawdown Duration                  394.0
Avg. Drawdown Duration              65.416667
# Trades                                 30.0
Win Rate [%]                        56.666667
Best Trade [%]                       4.264282
Worst Trade [%]                     -3.043164
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='21361', ...)

In [ ]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 0.05 #1000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.08

        if len(self.trades)==1:
            self.trades[-1].sl = self.trades[-1].entry_price

        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            sl1 = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]+sldiff*TPSLRatio
            tp2 = self.data.Close[-1]+sldiff
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            sl1 = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]-sldiff*TPSLRatio
            tp2 = self.data.Close[-1]-sldiff
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/100, commission=0.0000)
stat = bt.run()
stat

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_5560\2700601988.py:34: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    1125.0
Duration                               1125.0
Exposure Time [%]                   53.907638
Equity Final [$]                  42137.52524
Equity Peak [$]                   59205.63224
Return [%]                         321.375252
Buy & Hold Return [%]               47.141288
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -80.127035
Avg. Drawdown [%]                  -12.200691
Max. Drawdown Duration                  398.0
Avg. Drawdown Duration              27.162162
# Trades                                 12.0
Win Rate [%]                        66.666667
Best Trade [%]                      15.878988
Worst Trade [%]                     -8.401531
Avg. Trade [%]                    

In [ ]:
bt.plot()

Row(id='20190', ...)

In [ ]:
dfpl

,index,Open,High,Low,Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,EMASignal,ordersignal,pointposbreak
Gmt time,,,,,,,,,,,,,,,,
2017-11-29,287,263.738,266.027,263.658,264.927,188.5531,245.080210,97.447091,254.320110,259.34570,264.371290,3.875591,1.055288,0,0.0,NaN
2017-11-30,288,264.747,265.298,260.787,264.428,262.2805,245.228720,73.062405,254.060158,259.68670,265.313242,4.333331,0.921333,0,0.0,NaN
2017-03-12,291,266.307,266.787,264.147,264.168,209.1367,245.374335,57.950802,253.896049,259.97325,266.050451,4.675251,0.845122,0,0.0,NaN
2017-04-12,292,264.428,265.138,263.038,263.198,183.7908,245.508295,22.785845,253.910231,260.19230,266.474369,4.828789,0.739229,2,0.0,NaN
2017-05-12,293,262.848,263.718,262.718,263.208,167.5808,245.642950,23.739971,253.994826,260.41885,266.842874,4.933609,0.717087,2,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,1915,399.957,403.947,397.577,400.026,2496.7291,446.503915,61.313903,376.926390,416.16185,455.397310,18.855866,0.294372,1,0.0,NaN
2022-05-16,1916,406.506,408.367,402.566,408.227,2200.2466,446.357170,87.743486,378.565751,414.32485,450.083949,17.261383,0.414737,1,0.0,NaN
2022-05-17,1917,403.456,403.787,390.547,391.676,2204.2349,446.109920,23.351272,378.373766,411.67980,444.985834,16.180553,0.199697,1,0.0,NaN
